Dependencies

In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import datetime as dt

SQLAlchemy

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# Reflect existing database into a new model
#How to reflect two existing csv files brought into Pandas?

Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View classes found by automap
Base.classes.keys()


['measurement', 'station']

In [8]:
#Base.measurement.keys()

#Hawaii SQLite needs to be UTF-8 encoded?  See Stack Overflow response.

In [9]:
#Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [13]:
# Create session (link) from Python to the DB
session = Session(engine)

In [11]:
# # Use the Inspector to explore the database and print the table names
# inspector = inspect(engine)
# inspector.get_table_names()

In [ ]:
# Use Inspector to print the column names and types
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
#Just refer to meassurement and to station?  Or add in "hawaii"?
# Use `engine.execute` to select and display the first 10 rows from the measurement table
result = engine.execute("select *  from hawaii_measurements").fetchall()
print(result[:10])

In [ ]:
# Use `engine.execute` to select and display the first 10 rows from the station table
result = engine.execute("select *  from hawaii_stations").fetchall()
print(result[:10])

In [ ]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
first_row = session.query(Measurement).first()
first_row.__dict__

In [ ]:
# Find the most active stations?
active = session.query(Measurement).filter(Measurement.station).count()
print active

In [ ]:
# Use Inspector to print the column names and types
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Query Measurement for id`, `station`, date, prcp, tobs and `data` and save the query into results
id=[]
station=[]
date=[]
prcp=[]
tobs=[]
data=[]
for row in session.query(Measurement.id, Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs).all():
    id.append(row[0])
    station.append(row[1])
    date.append(row[2])
    prcp.append(row[3])
    tobs.append(row[4])

In [ ]:
engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()

Climate Analysis

In [1]:
# Calculate the total number of stations
session.query(Station.id).count()

NameError: name 'session' is not defined

In [ ]:
# Find the most active stations?
active = session.query(Measurement).filter(Measurement.station).count()
print active

In [ ]:
#List stations and observation counts in descending order
s_results = session.query(Measurement.station, func.count(Measurement.station)).\
            group_by(Measurement.station).\
            order_by(func.count(Measurement.station).desc()).all()
s_results

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
t_results = session.query(Measurement.station, Measurement.tobs).\
                filter(Measurement.station == best_station).\
                filter(Measurement.date >= last_twelve_months).all()
tobs_df = pd.DataFrame(t_results)
tobs_df.set_index('station', inplace=True)
tobs_df.head()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
best_station = s_results[0][0]
session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
                filter(Measurement.station == best_station).all()

In [ ]:
# # Earliest Date
# date_start = session.query(Measurement.date).order_by(Measurement.date).first().date
# date_start

In [ ]:
# # Latest Date
# latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first().date
# latest_date

In [ ]:
# Date 12 months from the latest date
last_twelve_months = dt.datetime.strptime(latest_date, '%Y-%m-%d') - dt.timedelta(days=365)
last_twelve_months

In [ ]:
# first_day = dt.datetime.strptime(latest_date, '%Y-%m-%d') - dt.timedelta(days=-730)
# first_day

In [ ]:
# Retrieve the last 12 months of temperature observation data
t_results = session.query(Measurement.date, func.avg(Measurement.tobs)).\
                    filter(Measurement.date >= last_twelve_months).\
                    group_by(Measurement.date).all()
t_results

In [ ]:
#Filter by station with highest number of observations
#Calculated above?

In [ ]:
# # Put data into dataframe
# temperature_df = pd.DataFrame(t_results, columns=['Date', 'Temperature'])
# temperature_df.set_index('Date', inplace=True)
# temperature_df.head()

In [ ]:
# Query the last 12 months of temperature observation data for this station
ax = temperature_df.plot(kind='bar', width=3, figsize=(12,8))
plt.locator_params(axis='x', nbins=6)
ax.xaxis.set_major_formatter(plt.NullFormatter())
ax.tick_params(axis='y', labelsize=16)
ax.grid(True)
plt.legend(bbox_to_anchor=(.3,1), fontsize="16")
plt.title("Temperature Last 12 Months", size=20)
plt.ylabel("Temperature (Degrees)", size=18)
plt.xlabel("Date", size=18)
plt.savefig("../Temperature.png")
plt.show

In [ ]:
# Plot the results as a histogram with bins=12.
tobs_df.plot.hist(by='station', bins=12, figsize=(12,8))
plt.grid()
plt.title("Temperature Observations for Station " + best_station, fontsize=20)
plt.xlabel("Temperature Reported", fontsize=16)
plt.legend(bbox_to_anchor=(1,1), fontsize=16)
plt.savefig("../StationTemps.png")
plt.show

In [ ]:
# # Print the summary statistics for the precipitation data
# precipitation_df.describe()

In [ ]:
# # Using the station id from the previous query, calculate the lowest temperature recorded, 
# # highest temperature recorded, and average temperature most active station?
# best_station = s_results[0][0]
# session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
#                 filter(Measurement.station == best_station).all()

In [ ]:
# # Plot the results as a histogram with bins=12.
# tobs_df.plot.hist(by='station', bins=12, figsize=(12,8))
# plt.grid()
# plt.title("Temperature Observations for Station " + best_station, fontsize=20)
# plt.xlabel("Temperature Reported", fontsize=16)
# plt.legend(bbox_to_anchor=(1,1), fontsize=16)
# plt.savefig("../StationTemps.png")
# plt.show

In [ ]:
#MAY NEED TO UPDATE DATES
# Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d 
# and return the minimum, average, and maximum temperatures for that range of dates.
def calc_temps(start_date, end_date):
    c_results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
                    filter(Measurement.date >= start_date).\
                    filter(Measurement.date <= end_date).all()
    return c_results
calc_temps('2017-01-01', '2017-12-31')

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
trip_results = calc_temps('2017-07-02', '2017-07-08')
trip_results

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
trip_df = pd.DataFrame(trip_results, columns=['Min Temp', 'Avg Temp', 'Max Temp'])
avg_temp = trip_df['Avg Temp']
min_max_temp = trip_df.iloc[0]['Max Temp'] - trip_df.iloc[0]['Min Temp']
avg_temp.plot(kind='bar', yerr=min_max_temp, figsize=(6,8), alpha=0.5, color='coral')
plt.title("Trip Avg Temp", fontsize=20)
plt.ylabel("Temp (F)")
plt.xticks([])
plt.grid()
plt.savefig("../TripTempSummary.png")
plt.show()